# Загрузка Pandas и очистка данных

In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind

In [ ]:
# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import random
import re
import datetime
from datetime import datetime
# Any results you write to the current directory are saved as output.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 45

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [2]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    df_output = df_input.copy()
    # убираем не нужные для модели признаки
    df_output.drop(['ID_TA',], axis = 1, inplace=True)
    # Заполняем "Number of Reviews" средними значениями
    df_output['Number of Reviews'] = df_output['Number of Reviews'].fillna(df_output['Number of Reviews'].mean())
     # Заполняем "Price Range" средними значениями (2)
    df_output['Price Range'].fillna(2, inplace=True)
    #Кодируем диапазоны цен 1,2,3. Пропуски заполняем значением 2, как наиболее популярным.
    df_output['Price Range'].replace('$','1',inplace=True)
    df_output['Price Range'].replace('$$ - $$$','2',inplace=True)
    df_output['Price Range'].replace('$$$$','3',inplace=True)
     # Добавляем столбец "Количество отзывов" ('Review_Count')
    df_output['Reviews Count'] = df_output.Reviews.apply(lambda x: len(str(x).split(' ')))    
    #Добавляем столбец "Количество ресторанов в городе" ('Rest_City_Count')
    df_output['Rest_City_Count'] = df_output.City.apply(lambda x: len(str(x)))
    # Добавляем столбец "Сетевой ли рестик" ('restaurant_chain_or_not')
    df_output['Restaurant_id'] = df_output['Restaurant_id'].apply(lambda x: x.split('_')[1])
    restaurant_chain_or_not = df_output[df_output['Restaurant_id'].isin(df_output['Restaurant_id'].value_counts()[df_output['Restaurant_id'].value_counts()>2].index)]
    restaurant_chain_or_not['restaurant_chain_or_not'] = '1'
    df_output['restaurant_chain_or_not'] = restaurant_chain_or_not['restaurant_chain_or_not']
    df_output['restaurant_chain_or_not'] = df_output['restaurant_chain_or_not'].fillna(0)
    # Добавляем столбец "Количество ресторанов в сети" ('Amount_Rest_network')
    test1 = df_output['Restaurant_id'].value_counts().rename_axis('Restaurant_id').reset_index(name='Amount_Rest_network')
    df_output = pd.merge(df_output, test1, how='right')
    # Добавляем столбец "Столица или нет" ('Capital_or_not')
    df_output['Capital_or_not'] = df_output['City']
    df_output.replace({'Capital_or_not' : { 'Paris' : '1', 'London' : '1',
                                           'Berlin' : '1', 'Rome' : '1',
                                           'Madrid' : '1', 'Prague' : '1',
                                           'Lisbon' : '1', 'Oslo' : '1', 'Warsaw':'1',
                                           'Amsterdam' : '1', 'Luxembourg' : '1',
                                           'Stockholm' : '1', 'Budapest' : '1',
                                           'Helsinki' : '1', 'Copenhagen' : '1',
                                           'Ljubljana' : '1', 'Bratislava' : '1',
                                           'Brussels' : '1', 'Athens' : '1',
                                           'Vienna' : '1', 'Edinburgh' : '1',
                                           'Geneva' : '1', 'Munich' : '0', 'Dublin':'1',
                                           'Oporto' : '0', 'Zurich' : '0',
                                           'Milan' : '0', 'Barcelona' : '0',
                                           'Lyon' : '0', 'Hamburg' : '0',
                                           'Krakow' : '0'}},inplace=True)
    # Добавляем столбец "Численность Население" ('People_quontity')  
    
    people_quontity = {
    'London' : 8909081,'Paris' : 2148271,'Madrid' : 3223334,'Barcelona' : 1620343,
    'Berlin' : 3769495,'Milan' : 1399860,'Rome' : 2860009,'Prague' : 1324277,
    'Lisbon' : 505526,'Vienna' : 2600000,'Amsterdam' : 872680,'Brussels' : 1208542,
    'Hamburg' : 1845229,'Munich' : 1484226,'Lyon' : 516092,'Stockholm' : 975904,
    'Budapest' : 1752286,'Warsaw' : 1793579,'Dublin' : 554554,'Copenhagen' : 794128,
    'Athens' : 664046,'Edinburgh' : 488050,'Zurich' : 415215,'Oporto' : 287591,
    'Geneva' : 201818,'Krakow' : 779115,'Oslo' : 697549,'Helsinki' : 656229,
    'Bratislava' : 437726,'Luxembourg' : 626108,'Ljubljana' : 295504
    }
    df_output['People_quontity'] = df_output['City'].map(people_quontity)
    
    # Добавляем столбец "Плотность Население" ('People_density')  
    #df_output['People_density'] = df_output['City']
    #df_output.replace({'People_density' : {
    people_density = {
    'Paris' : 20781,'London' : 5667,'Berlin' : 4088, 'Rome' : 2234,
    'Madrid' : 5380,'Prague' : 2506,'Lisbon' : 5093, 'Oslo' : 1483, 
    'Warsaw': 3449, 'Amsterdam' : 4768,'Luxembourg' : 2240,  'Stockholm' : 5140,
    'Budapest' : 3331,  'Helsinki' : 3071,'Copenhagen' : 6215,'Ljubljana' : 1736,
    'Bratislava' : 1189, 'Brussels' : 5497,'Athens' : 7500,'Vienna' : 4503,
    'Edinburgh' : 4140,'Geneva' : 12589,'Munich' : 4713,'Dublin': 3689,                    
    'Oporto' : 5703,'Zurich' : 4666, 'Milan' : 7589,'Barcelona' : 16641,                          
    'Lyon' : 10041,'Hamburg' : 2388, 'Krakow' : 2384
    }
    df_output['People_density'] = df_output['City'].map(people_density)
    
    # Добавляем столбец "Name_city_01" ('10-ричный код названия города')  
    df_output['Name_city_01'] = df_output['City']
    df_output.replace({'Name_city_01' : { 'Paris' : '5774918292', 'London' : '538887778887',
                                           'Berlin' : '437891858287 ', 'Rome' : '59888678',
                                           'Madrid' : '3266126', 'Prague' : '579174809478',
                                           'Lisbon' : '538292758887', 'Oslo' : '56928588', 'Warsaw':'647491927496',
                                           'Amsterdam' : '428692937891777486', 'Luxembourg' : '53949778867588949180',
                                           'Stockholm' : '609388768481888586', 'Budapest' : '4394777489789293',
                                           'Helsinki' : '4978859282878482', 'Copenhagen' : '44888978878174807887',
                                           'Ljubljana' : '538394758583748774', 'Bratislava' : '43917493829285749574',
                                           'Brussels' : '4391949292788592', 'Athens' : '429381788792',
                                           'Vienna' : '638278878774', 'Edinburgh' : '467782877594918081',
                                           'Geneva' : '487887789574', 'Munich' : '549487827681', 'Dublin':'459475858287',
                                           'Oporto' : '568988919388', 'Zurich' : '679491827681',
                                           'Milan' : '5482857487', 'Barcelona' : '437491767885888774',
                                           'Lyon' : '53988887', 'Hamburg' : '49748675949180',
                                           'Krakow' : '529174848896'}},inplace=True)
     #Добавляем столбец "Покупательская способность" ('by_PI')  
    by_PI = {
    'London' : 24, 'Paris' : 22, 'Madrid' : 23, 'Barcelona' : 30, 'Berlin' : 8,
    'Milan' : 40, 'Rome' : 37, 'Prague' : 31, 'Lisbon' : 48, 'Vienna' : 21,
    'Amsterdam' : 19, 'Brussels' : 15, 'Hamburg' : 7, 'Munich' : 6,'Lyon' : 70,
    'Stockholm' : 13, 'Budapest' : 51, 'Warsaw' : 29, 'Dublin' : 27, 'Copenhagen' : 10,
    'Athens' : 53, 'Edinburgh' : 16, 'Zurich' : 2, 'Oporto' : 42, 'Geneva' : 3, 
    'Krakow' : 36, 'Oslo' : 17, 'Helsinki' : 9, 'Bratislava' : 39, 'Luxembourg' : 4,
    'Ljubljana' : 32}

    df_output['by_PI'] = df_output['City'].map(by_PI)
    #Узнаем параметр "ресторан на человека" "Rest_per_human"
    df_output['Rest_per_human'] = df_output['Rest_City_Count']/(df_output['People_quontity'])
    # Узнаем как часто оставляются отзывы.
    df_output['Num_per_Pq'] = df_output['Number of Reviews'] / df_output['People_quontity']
    #для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
    df_output = pd.get_dummies(df_output, columns=[ 'City'], dummy_na=True)
    ####ДАТЫ ДАТЫ ДАТЫ
    pattern = re.compile('\'\d+\/\d+\/\d+\'?')

    df_output['reviews_date'] = df_output['Reviews'].apply(lambda x: pattern.findall(str(x)))
    df_output['reviews_date'] = df_output['reviews_date'].apply(
    lambda x: str(x).replace('[', '').replace("'", '').replace(']', '').replace(' ', '').replace('"', '').split(','))

    df_output['fist_reviews'] = df_output['reviews_date'].apply(lambda x: x[0])
    df_output['last_reviews'] = df_output['reviews_date'].apply(lambda x: x[-1])

    df_output['fist_reviews'] = df_output['fist_reviews'].apply(
    lambda x: x.replace('[', '').replace("'", '').replace(']', '').replace(' ', '').replace('"', ''))
    df_output['last_reviews'] = df_output['last_reviews'].apply(
    lambda x: x.replace('[', '').replace("'", '').replace(']', '').replace(' ', '').replace('"', ''))
    df_output['fist_reviews'] = df_output['fist_reviews'].apply(lambda x: x if len(x) == 0 else pd.to_datetime(x))
    df_output['last_reviews'] = df_output['last_reviews'].apply(lambda x: x if len(x) == 0 else pd.to_datetime(x))
    df_output['d1'] = (df_output['fist_reviews'] - datetime(2004, 1, 1, 0, 0, 0)).dt.days
    df_output['d2'] = (df_output['last_reviews'] - datetime(2007, 1, 1, 0, 0, 0)).dt.days
    # Заполняем 'd1' , 'd1' средними значениями
    df_output['d1'] = df_output['d1'].fillna(df_output['d1'].mean())
    df_output['d2'] = df_output['d2'].fillna(df_output['d2'].mean())
     # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
     # object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
     # df_output.drop(object_columns, axis = 1, inplace=True)
    df_output=df_output.drop(['Restaurant_id','Cuisine Style','Reviews','URL_TA','reviews_date','fist_reviews','last_reviews'], axis = 1)
    return df_output

,Restaurant_id,City,Cuisine_Style,Ranking,Rating,Price_Range,Number_of_Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,2,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,0,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,3,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,0,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,2,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963
...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,2,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414
39996,id_6340,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,2,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036
39997,id_1649,Stockholm,"['Japanese', 'Sushi']",1652.0,4.5,0,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615
39998,id_640,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,2,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838


In [ ]:
#Запускаем и проверяем что получилось
df_preproc.info()

In [392]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = test1_X.drop(['Restaurant_id', 'Rating'], axis = 1)
y = test1_X['Rating']

In [393]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [394]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [395]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [396]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [398]:
4
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.22136749999999997
